In [1]:
import pandas as pd 
import numpy as np

import nltk
from nltk.corpus import stopwords  
from nltk import word_tokenize    
import re
import pymorphy2
from tqdm.auto import tqdm, trange 
import docx
import os

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

%matplotlib inline

### Получим данные с текстом сообщения и номером категории, загрузим это в датафрейм

In [2]:
entries = os.scandir('Документы/')

columns = list([i for i in range(1,40)])
data = dict((k, []) for k in columns)
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)
countFile = 0
for entry in entries:
    # Debug
    #if entry.name != '208_ot_18_fevralya_2022.docx' and entry.name != '804_ot_30_aprelya_2022.docx':
    #    continue
    countFile += 1

    #print(str(countFile) + ':' + entry.name)

    raw_text = getText(entry)

    # get text outside tags
    #convert_text = re.split('(?<=\{11\}).*(?=\{11\})',raw_text)
    #convert_text = [string.upper() for string in convert_text]

    # get text inside tags with tags
    dateJoin = pd.DataFrame() # initialize empty df
    for i in columns:
        remaining_parts = re.findall('(?<=\{' + i.__str__() + '\}).*(?=\{' + i.__str__() + '\})', raw_text)
        if len(remaining_parts) == 0:
            continue      
        data[i] += (remaining_parts)

df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in data.items() ]))
df.replace('|','\\')
df.to_csv('result.csv', encoding='utf-8', index=False, sep='|')

Создадим новый датафремй с 2 колонками - категорией и текстом сообщения.

In [4]:
df1 = df.melt().dropna().rename({'value':'text', 'variable': 'label'},axis=1).reset_index()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17281 entries, 0 to 17280
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   17281 non-null  int64 
 1   label   17281 non-null  int64 
 2   text    17281 non-null  object
dtypes: int64(2), object(1)
memory usage: 405.1+ KB


Поработаем дальше с текстом, создав колонку со словами текста, приведенными к словарной форме

In [5]:
# новая колонка, пока что заполненная нулями
df1['words'] = np.zeros

# регулярка для поиска русских слов
regular_expr = '[а-я]+'
reg_expr_compiled = re.compile(regular_expr)

# создадим экземпляр класа MorphAnalyzer для лемматизации (приведения слов к словарной форме)
morph = pymorphy2.MorphAnalyzer()
# Стоп-слова 
rus_stopwords = stopwords.words("russian") 
# Расширение списка стоп-слов (см. набор данных)
rus_stopwords.extend(['т.д.', 'т', 'д', 'который', 'российская', 'федерация'])

for index, i_text in enumerate(df1['text']):
    # приводим текст к нижнему регистру
    i_text = i_text.lower()
    # разбиваем текст на слова 
    i_text = ' '.join(reg_expr_compiled.findall(i_text))
    # создаем список, чтобы добавлять в него нормализованные слова
    norm_words = list()
    for i_word in i_text.split():
        #нормализуем слово
        i_word = morph.parse(i_word)[0].normal_form
        if i_word not in rus_stopwords:
            norm_words.append(i_word)
    # Строка с нормализованными словами
    i_text = ' '.join(norm_words)
    # Меняем значение колонки с 0 на полученную строку
    df1.loc[index, 'words']  = i_text
    
df1.head(5)

,index,label,text,words
0,0,1,"Понятия, используемые в настоящих Правилах, оз...",понятие использовать настоящий правило означат...
1,1,1,"""вознаграждение"" - процентные отчисления, выпл...",вознаграждение процентный отчисление выплачива...
2,2,1,"""исполнитель"" - российская организация, осущес...",исполнитель российский организация осуществлят...
3,3,1,"""исполнитель по разработке стандартных образцо...",исполнитель разработка стандартный образец рос...
4,4,1,"""квалификация"" - процедура отбора, проводимая ...",квалификация процедура отбор проводить операто...


Подготовим данные для моделей обучения.
Разобъем наш датасет на обучающий и валидационный. В матрице объектов-признаков оставим только колонку "words", полученную нами после работы с признаком "text"

In [6]:
X = df1['words']
y = df1['label']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 17)

In [8]:
# проверяем, что все категории попали в валидационный датасет
print(len(y_valid.unique()))
y_valid.unique()

39


array([35, 13, 36,  2, 26, 10, 11,  3, 15, 19,  1, 31, 37, 27, 21, 24, 22,
        5, 30, 20, 28, 12,  4, 38, 17,  9, 33, 25,  7,  8, 32, 29,  6, 18,
       23, 39, 16, 34, 14], dtype=int64)

## Построение моделей классификации и получение первых результатов проверки качества

Для классификациии текста воспользуемся 4 моделями: мультиномиальным наивным байесовским классификатором, логистической регрессией, методом k-ближайших соседей, стохастическим градиентным спуском. Также для автоматизации предварительных преобразований данных перед обучением модели будем использовать класс Pipeline, чтобы упростить работу преобразования текста в числовые объекты. Для данного преобразования будут использоваться модель "мешка слов", для чего потребуется класс CountVectorizer, чтобы преобразовать текст в вектор на основе частоты (количества) каждого слова, встречающегося во всем тексте. Потребуется и взвешивание важности слов (TF-IDF), что поможет реализовать класс TfidfTransformer, который нужен для преобразования текста в частотные векторы слов. 

### Обучим и получим предсказание от линейной регрессии

In [9]:
lr_model = Pipeline(steps = [('vect', CountVectorizer(ngram_range = (1, 1))),
                       ('tfidf', TfidfTransformer(use_idf=True)),
                       ('lr_clf', LogisticRegression(random_state = 17,
                                                    C = 9.0,
                                                    class_weight = 'balanced',
                                                    multi_class = 'auto',
                                                    solver = 'saga',
                                                    l1_ratio = 0.5,
                                                    penalty = 'elasticnet',
                                                    max_iter = 200,
                                                    n_jobs = 6))
                             ]
                   )

lr_model = lr_model.fit(X_train, y_train)

C:\Users\oluny\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [10]:
lr_pred = lr_model.predict(X_valid)

In [19]:
accuracy_1 = accuracy_score(y_valid, lr_pred)
accuracy_1

0.5756436216372577

### Обучим и получим предсказание от наивного байеса

In [21]:
nb = Pipeline(steps = [('vect', CountVectorizer(ngram_range = (1, 1))),
               ('tfidf', TfidfTransformer(use_idf = True)),
               ('nb_clf', MultinomialNB(alpha= 0.9)) 
                        ]
                       )
                       

nb_model = nb.fit(X_train, y_train)
nb_pred = nb_model.predict(X_valid)
accuracy_2 = accuracy_score(y_valid, nb_pred)
accuracy_2

0.5131617008967313

### Воспользуемся методом k-ближайших соседей и получим предсказание, результат метрики

In [16]:
knn = Pipeline(steps = [('vect', CountVectorizer(ngram_range = (1, 1))),
                        ('tfidf', TfidfTransformer(use_idf = True)),
                        ('knn_clf', KNeighborsClassifier(metric='minkowski',
                                                         n_neighbors=10, 
                                                         weights ='distance'))
                       ]
              )
knn_model = knn.fit(X_train, y_train)
knn_pred = knn_model.predict(X_valid)
accuracy_3 = accuracy_score(y_valid, knn_pred)
accuracy_3

0.5160543824124963

### Воспользуемся стохастическим градиентным спуском и получим предсказание

In [14]:
sgd = Pipeline(steps = [('vect', CountVectorizer(ngram_range = (1, 2))),
                        ('tfidf', TfidfTransformer(use_idf = True)),
                        ('sgd_clf', SGDClassifier(random_state = 17,
                                                 alpha= 0.000012, 
                                                  class_weight = None, 
                                                  loss= 'hinge', 
                                                  penalty='elasticnet',
                                                  l1_ratio= 0.1601,
                                                  max_iter = 3000))
                         ]
              )
sgd_model = sgd.fit(X_train, y_train)
sgd_pred = sgd_model.predict(X_valid)
accuracy_4 = accuracy_score(y_valid, sgd_pred)
accuracy_4

0.6603991900491756

Посмотрим на матрицу ошибок, поймем, какие категории лучшая модель - Логистическая регрессия - предсказывает хорошо, а где чаще ошибается

In [123]:
pd.set_option('display.max_columns', None)

pd.DataFrame(confusion_matrix(y_valid, sgd_pred), index = range(1, 40))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
1,106,3,1,0,1,3,0,0,0,1,2,4,0,0,0,0,0,0,0,0,0,4,0,4,0,0,2,0,0,2,0,0,0,0,3,0,0,0,0
2,3,86,1,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,5,0,1,0,0,4,1,0,6,0,0,0,0,2,0,0,0,0
3,0,0,16,2,0,0,0,0,0,4,0,0,0,0,5,1,0,0,1,2,0,2,1,3,0,0,0,1,1,0,0,0,0,0,2,1,1,2,0
4,1,6,3,17,1,0,0,1,0,0,2,1,0,0,0,0,0,0,0,0,0,6,0,4,0,0,2,1,4,3,0,0,0,0,2,0,0,0,0
5,1,1,0,0,1,1,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,4,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
6,1,0,1,2,0,9,0,2,0,2,5,2,0,0,2,0,0,0,2,0,0,2,0,4,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0
7,0,0,0,2,0,0,33,0,1,7,0,0,1,0,5,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0
8,0,0,1,0,0,3,0,7,0,0,0,0,0,0,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,2,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
10,1,0,0,0,0,0,3,0,1,194,1,1,1,0,8,0,1,0,9,0,0,0,0,5,0,0,1,0,0,0,0,0,0,0,10,2,0,2,0


In [112]:
classification_report(y_valid, sgd_pred)

'              precision    recall  f1-score   support\n\n           1       0.72      0.78      0.75       136\n           2       0.77      0.76      0.77       113\n           3       0.55      0.36      0.43        45\n           4       0.37      0.31      0.34        54\n           5       0.25      0.07      0.11        15\n           6       0.32      0.24      0.28        37\n           7       0.62      0.60      0.61        55\n           8       0.50      0.47      0.48        15\n           9       0.50      0.29      0.36         7\n          10       0.76      0.81      0.79       240\n          11       0.77      0.82      0.80       259\n          12       0.51      0.42      0.46        84\n          13       0.38      0.19      0.25        53\n          14       0.60      0.38      0.46         8\n          15       0.54      0.60      0.57       146\n          16       0.81      0.67      0.73        43\n          17       0.71      0.66      0.68        73\n       